# Exercise 1: Protobuf

## Part 1: Warm Up

Below is an excerpt from `warm_up.proto`.

```
// Defines the details of a fish.
message Fish {
  required string species = 1;
  optional float size_in_cm = 2;
}

// Defines an aquarium which can contain multiple fishes.
message Aquarium {
  repeated Fish fishes = 1;
}
```
### Using protobuf (tutorial)

Let's take a brief look at how python uses protobuf definitions and protoc, the protobuf compiler, to send structured data between different services or applications in a language- and platform-neutral format. The data structures defined in the .proto file are compiled into python classes. Protobuf's library functions are then used to serialize and deserialize the data.

In the snip below we use the jupyter notebook magic command `%cd` to move to the `warm_up` directory and `!` to call a shell command in a new process (similar to `fork()` and `exec()`). Protoc creates a `.py` file used for the definition, serialization, and deserialization of protobuf-defined types.

In [ ]:
%cd warm_up

In [ ]:
!python -m grpc_tools.protoc warm_up.proto --proto_path=. --python_out=. 

Inspect `warm_up_pb2.py`. Although not represented by traditional classes, protobuf creates metaclasses for the different message types. These metaclass structures work as templates for classes using the protobuf definition, e.g., warm_up_pb2.Fish in the following cell. We can define messages as python objects by importing `warm_up_pb2.py` and building the fields using the constructor:

In [ ]:
import warm_up_pb2

my_fishie = warm_up_pb2.Fish(species="Clown Fish", size_in_cm=10)
print(my_fishie)

: 

This works the same way for nested and repeated messages. Run the code block below, then try adding a new fish object and adding it to the aquarium.

In [ ]:
my_aquarium = warm_up_pb2.Aquarium(fishes=[my_fishie])
print(my_aquarium)

Lastly, we can also access and set attributes of message objects using the dot operator. Try modifying the size of your fish object by accessing the nested attributes of `my_aquarium`.

In [ ]:
my_fishie.size_in_cm = 12
print(my_fishie)

See this [protobuf python tutorial](https://protobuf.dev/getting-started/pythontutorial/) and the [proto2 language guide](https://protobuf.dev/programming-guides/proto2/) for additional information about using protobuf.

**Question 1**: Use the procedure seen in lecture to encode a `Fish` message it into its serialized binary representation, and then convert it into hexadecimal. Use the values `Clown Fish` and `10` for the `species` and `size_in_cm` fields, respectively. Show your steps.

## Part 2: Scenario

### Background
You are a systems engineer at Stellar Horizons, a cutting-edge space startup that specializes in interplanetary missions. Recently, your team successfully launched the Jupyter Probe to study the atmosphere of Jupiter.

During routine operations, communication with Jupyter Probe was suddenly lost due to a power surge and system failure at the Ares Link Facility, which serves as the primary communication hub for the probe.

As the lead systems engineer, your responsibility is to manage the crisis. The fate of the Jupyter Probe, a multi-million-dollar investment, hangs in the balance. Preliminary analysis suggests the last data transmission might contain clues about the probe’s current status and trajectory.

### Part 2.1: Data Analysis
As part of your contingency operations, you access the pcapng (Packet Capture) file containing the last transmissions received from the Jupyter Probe. These transmissions are encoded using Protocol Buffers. However, the protobuf definition file was corrupted when the system went down. Your team is currently working to recover it but in the meantime you must parse out what information you can from the packet logs.

**Question 2**: open the file in Wireshark to determine the following:
a. What transport protocol is the system using to send and receive messages?
b. There are two port numbers being used, what are they? Which is the Ares Link ground station and which is the Jupyter Probe?
c. How many message types do you see?
d. Decode the first instance of each message type to get the following information about each field:
	1. Field number
	2. Field type
	3. Field value (decode varint and string types, leave floating point values as binary)


### Part 2.2: Regaining Control
In a fortunate turn of events, an intern at the facility has recovered a copy of the protobuf specification used for communications with Jupyter Probe (`specification.txt`). This document provides the names of the variables encoded in the transmissions, which will aid in correlating them with the data you've recovered.

The flight dynamics team has reported that the initial data decoded from the pcapng file indicates that Jupyter Probe is currently in an unstable orbit around Mars. If immediate corrective actions are not taken, the probe could either escape Mars's orbit on a bad trajectory or crash into its surface.

You must bootstrap a solution. As a minimum, the flight dynamics team has identified that sending a packet with correctly calculated **engine burn time** and **angle of thrust** floating point values should command the probe to correct its course. Thinking back to your time as an EEE466 student, you know it's possible to develop a python script that can interpret the incoming telemetry and send new flight commands to adjust back to a safe trajectory using protobuf marshalling. The flight dynamics team will provide you with the trajectory equation based on the current orbital parameters and the desired trajectory.

Question 3: Using the data you decoded from the packet logs and hints from the specification file, write a protobuf file that will allow you to interface with Jupyter Probe.

Question 4: Write a ground station python script to accept telemetry data from Jupyter Probe, perform the adjustment calculation, and transmit a control signal. Listen for new data to come back from the probe indicating that it has returned to a safe trajectory.

The desired velocity for a safe trajectory is x=3,y=1,z=0 km/s. The engineers at flight dynamics have represented this as an array:

```
desired_velocity = np.array([3, 1, 0]) 
```

You should use the same format after receiving the current velocity from Jupyter Probe.

When you're ready to test your solution, run `jupyter_probe.exe`. Show that you have completed the challenge by printing the velocity data and the message containing the secret word.


## Submission
Submit a zip archive in the format `eee446_ex1_lastname1_lastname2.zip` that includes the following:
1. This `.ipynb` notebook with the answers to questions 1 and 2 appended as a new cell.
2. Your proto file (question 3).
3. Your ground station python file (question 4).

There is no report required for this exercise.

###Question 1:

### My Answer:

The message would start with the fish:
tag: continuation bit: 0 ; tag #: 1 ; tag type: 2 (string)
0000 1010 
data: first bit is length (10 in hex), followed by 10 bytes of data (Clown Fish)
0A 43 6C 6F 77 6E 20 46 69 73 68

tag: continuation bit: 0 ; tag #: 2 ; tag type: 0 (varint)
0001 0000
data: 1 byte: continuation bit = 0
0000 1010

Final serialized binary in hex:

0A 0A 43 6C 6F 77 6E 20 46 69 73 68 10 0A

### End of My Answer

###Question 2

### My Answer:
a: what transport protocol is being used? UDP is being used.

b: what are the port numbers? Port 64037 (Probe) and port 50320 (ground station).

c: There are 3 message types: one is a string message which is sent from the probe to the ground station, another is a string + data message containing the x, y, and z values of the probe, and the last is a data message from the ground station to the probe which holds the burn time and angle of thrust.

d: First message type:
Tag: continuation bit: 0 ; tag #: 1 ; tag type: 2 (string)
Data part 1: Number of bytes to follow: 53
Data part 2: The string "Hello! This is Jupyter Probe on a mission to Jupiter."

Second message type:
Tag: Continuation bit: 0 ; tag #: 1 ; tag type: 2 (string)
Data: len: 46 ; data: Transmission of velocity data in x, y, z axes.

Tag: Continuation bit: 0 ; tag #: 2 ; tag type: 2 (embedded message)
Data: len: 6 ; data: 8  4  16  0  24  1

Third message type:
Tag: Continuation bit: 0 ; tag #: 1 ; tag type: 5 (i32)
Data: 13 86 27 43, or 00010011 10000110 00100111 01000011. This is encoded in little endian encoding. Swapped around, this number is 01000011001001110110100000010011, or 167.4065399169921875.
Tag: Continuation bit: 0 ; tag #: 2 ; tag type: 5 (i32)
Data: 3e da 3f 42, or 00111110 11011010 00111111 01000010. This is encoded in little endian encoding. Swapped around, this number is 01000010001111111101101000111110, or 47.96312713623046875.

This float converter was used for the calculations: https://www.h-schmidt.net/FloatConverter/IEEE754.html

This hex - binary - decimal converter was used: https://www.rapidtables.com/convert/number/hex-to-binary.html?x=423fda3e

###End of My Answer